In [13]:
import pandas as pd
import numpy as np
import sys
import re
from io import StringIO, BytesIO
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

In [14]:
codestring = '''test_df = pd.DataFrame(np.random.randn(1000, 4), index=pd.date_range('1/1/2000', periods=1000), columns=list('ABCD'))
test_df.head()'''
code2 = '''# this is a test of multiline code including indented lines
def test():
    print('this is a test')
    
test()'''
code3 = '''def test2(oh, my):
    if oh == my:
        return 'same outputs'
    else:
        return 'different outputs'

oh = 'oh'
my = 'my'

test2(oh, my)'''
code4 = '''a = 10
b = 20
c = a + b
c'''

In [ ]:
import logging
import contextlib

# write a context manager that redirects stdout, stderr, and any other caught outputs to a variable
class OutputLogger:
    def __init__(self, name="root", level=)

In [78]:
import re
import contextlib

# analyze a string of code and determine whether it contains potentially malicious code. Returns False for potentially malicious code, True for supposedly safe code
def check_code(code):
    code = re.sub(r'\s{2,}', ' ', code)
    forbidden = ['os.system', 'subprocess.call', 'from os import system', 'import subprocess', 'from subprocess import call', 'shell=True', 'shell = True', 'shell= True', 'shell =True']
    for f in forbidden:
        if f in code:
            return False
        
    return True


# split a string of code into blocks of code based on newline unless a line starts with whitespace
def split_to_blocks(code):
    codeblocks = []
    for line in code.split('\n'):
        if len(line.strip()) == 0:
            continue
        elif line.startswith(' ') or line.startswith('\t'):
            codeblocks[-1] += '\n' + line
        else:
            codeblocks.append(line)
    return codeblocks


# analyze blocks generated from user code and determine run schema
def run_schema(blocks):
    if len(blocks) == 1:
        return 'single'
    elif len(blocks) >= 2 and len(blocks[-1].splitlines()) <= 2:
        return 'multi_last_single'
    elif len(blocks) >= 2 and len(blocks[-1].splitlines()) > 2:
        return 'multi_last_multi'
    else:
        return 'error'


# run code blocks based on run schema
def run_code(blocks, schema, namespace):
    if schema == 'single':
        ccode = compile(blocks[0], '<string>', 'single')
        
        exec(ccode, namespace)
    elif schema == 'multi_last_single':
        first = '\n'.join(blocks[:-1])
        ccode1 = compile(first, '<string>', 'exec')
        exec(ccode1, namespace)
        ccode2 = compile(blocks[-1], '<string>', 'single')
        exec(ccode2, namespace)
    elif schema == 'multi_last_multi':
        first = '\n'.join(blocks[:-1])
        ccode1 = compile(first, '<string>', 'exec')
        exec(ccode1, namespace)
        ccode2 = compile(blocks[-1], '<string>', 'exec')
        exec(ccode2, namespace)


# all-in-one function for parsing and running code
def code_parse(code):
    status = check_code(code)
    if status == False:
        raise Exception('The provided code does not seem to be secure. Aborting process...')
    else:
        blocks = split_to_blocks(code)
        schema = run_schema(blocks)
        

In [21]:
blocks = split_to_blocks(code4)

schema = run_schema(blocks)

namespace = {}
run_code(blocks, schema, namespace)

30

In [26]:
df = pd.DataFrame(np.random.randn(1000, 4), index=pd.date_range('1/1/2000', periods=1000), columns=list('ABCD'))

In [107]:
import subprocess
from io import StringIO

cmd = [sys.executable, '-u', '-c', '''import pandas as pd
df = pd.DataFrame({"a": [1,2,3], "b": [9,8,7]})
print(df.head())''']
a = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, bufsize=1)
out, err = a.communicate()

In [110]:
out.decode('utf-8')

'   a  b\n0  1  9\n1  2  8\n2  3  7\n'

In [109]:
err

b''

In [120]:
c1 = '''import pandas as pd
df = pd.DataFrame({"a": [1,2,3], "b": [9,8,7]})'''
c2 = '''df.head()'''

In [118]:
exec(compile(c1, '<string>', 'exec'))

In [124]:
old_stdout = sys.stdout

In [125]:
new_output = StringIO()
sys.stdout = new_output
exec(compile(c2, '<string>', 'single'))
sys.stdout = old_stdout

,a,b
0,1,9
1,2,8
2,3,7


In [126]:
print(new_output.getvalue())